#### Imports/setup

In [1]:
from parameters.config_imports import *

searching for user's gee cloud project name for account linked to sepal. i.e., a python file located here: 'parameters/config_gee.py', containing: gee_cloud_project 'insert_project_name_here' 
Earth Engine has been initialized with the specified credentials.


In [2]:
def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out

#### Inputs : feature collection (without Geo IDs)
- Choose between 1) GEE asset or 2) Geojson input
- Comment out the one you dont want

In [3]:
# ## 1 - GEE asset
# asset_id = "projects/ee-whisp/assets/example_asset" # asset id
# roi = ee.FeatureCollection(asset_id) # load feature collection asset

# 2 - Geojson input
geojson_path = "input_examples/geojson_example.geojson" # geojson file
roi = geojson_to_ee(geojson_path) # convert geojson to feature collection

In [4]:
roi # view feature collection properties

In [5]:
# optional code: uncomment below line to test converting to point inputs using polygon centroid

# roi = roi.map(lambda feature: ee.Feature(feature.geometry().centroid(1))) 

### Whisp it
- If over 500 polygons currently this will go to Google Drive and require manual uploading here
- This value can be changed in parameters.config_runtime.py 
- NB Currently stats processing sent to a separate notebook 'process_whisp_stats.pynb'
- If your GeoJSON already holds GeoIDs, make sure that the GeoID column is labeled **"geoid"**

In [6]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
Datasets present in lookup:  True
geo id not present ['Plot_ID', 'Plot_area_ha', 'Geometry_type', 'Country', 'Admin_Level_1', 'Centroid_lon', 'Centroid_lat', 'In_waterbody', 'Unit']
...stats processing started
...processing time: 5.352310657501221 seconds


### Display it


In [7]:
df

,Plot_ID,Plot_area_ha,Geometry_type,Country,Admin_Level_1,Centroid_lon,Centroid_lat,In_waterbody,Unit,EUFO_2020,...,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,RADD_before_2020,TMF_deg_after_2020,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index
0,1,1.938,Polygon,GHA,Ashanti Region,-1.611942,6.159539,-,ha,0.000,...,1.552,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0
1,2,4.153,Polygon,GHA,Ashanti Region,-1.644732,6.104735,-,ha,0.361,...,2.724,0.000,0.000,0.000,0.000,0.089,0.743,0.000,0.000,1
2,3,16.602,Polygon,GHA,Western Region,-2.157144,5.981149,-,ha,16.602,...,0.000,0.000,0.000,1.111,0.089,0.000,0.000,0.000,1.063,2
3,4,31.210,Polygon,IDN,South Sumatra,103.956096,-3.054668,-,ha,0.000,...,31.020,31.210,6.401,0.000,0.000,0.000,0.000,0.000,0.000,3
4,5,1.964,Polygon,IDN,South Sumatra,103.970372,-3.068831,-,ha,0.000,...,1.950,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,4
5,6,12.723,Polygon,IDN,South Sumatra,103.975182,-3.082922,-,ha,7.998,...,1.899,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5
6,7,20.884,Polygon,IDN,South Sumatra,103.977512,-3.083808,-,ha,6.017,...,9.043,0.000,0.000,0.000,0.714,0.000,0.000,0.000,0.000,6
7,8,8.279,Polygon,CIV,Lagunes,-4.101646,5.711935,-,ha,0.404,...,3.194,0.000,0.000,0.000,0.000,0.000,1.602,0.000,0.001,7
8,9,1.981,Polygon,CIV,Lagunes,-4.086848,5.673810,-,ha,0.000,...,0.435,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,8
9,10,3.798,Polygon,CIV,District Autonome D'Abidjan,-4.119589,5.572136,-,ha,0.000,...,1.162,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9


#### Export it to CSV file
NB if exporting to Google Drive skip this and upload to your results folder 'whisp_output_table.csv'

In [8]:
#export output csv (default: '/home/sepal-user/module_results/whisp/whisp_output_table.csv')
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)

output csv:  /home/sepal-user/module_results/whisp/whisp_output_table.csv


##### Calculate EUDR risk category

In [9]:
# Replace with name of your CSV file 
# NB if skipped 'Add Geo ids' should set to "out_directory/whisp_output_table.csv" 
csv_file_input =out_directory/'whisp_output_table.csv'

# import csv as a dataframe
df_w_stats = pd.read_csv(csv_file_input)

df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )


##### Display table with risk columns

In [10]:
df_w_risk

,Plot_ID,Plot_area_ha,Geometry_type,Country,Admin_Level_1,Centroid_lon,Centroid_lat,In_waterbody,Unit,EUFO_2020,...,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,1,1.938,Polygon,GHA,Ashanti Region,-1.611942,6.159539,-,ha,0.000,...,0.000,0.000,0.000,0.000,0,yes,yes,yes,no,low
1,2,4.153,Polygon,GHA,Ashanti Region,-1.644732,6.104735,-,ha,0.361,...,0.089,0.743,0.000,0.000,1,yes,no,yes,yes,low
2,3,16.602,Polygon,GHA,Western Region,-2.157144,5.981149,-,ha,16.602,...,0.000,0.000,0.000,1.063,2,yes,no,yes,yes,low
3,4,31.210,Polygon,IDN,South Sumatra,103.956096,-3.054668,-,ha,0.000,...,0.000,0.000,0.000,0.000,3,yes,yes,yes,no,low
4,5,1.964,Polygon,IDN,South Sumatra,103.970372,-3.068831,-,ha,0.000,...,0.001,0.000,0.000,0.000,4,yes,yes,yes,yes,low
5,6,12.723,Polygon,IDN,South Sumatra,103.975182,-3.082922,-,ha,7.998,...,0.000,0.000,0.000,0.000,5,yes,no,yes,no,low
6,7,20.884,Polygon,IDN,South Sumatra,103.977512,-3.083808,-,ha,6.017,...,0.000,0.000,0.000,0.000,6,yes,yes,yes,yes,low
7,8,8.279,Polygon,CIV,Lagunes,-4.101646,5.711935,-,ha,0.404,...,0.000,1.602,0.000,0.001,7,yes,yes,yes,yes,low
8,9,1.981,Polygon,CIV,Lagunes,-4.086848,5.673810,-,ha,0.000,...,0.000,0.000,0.000,0.000,8,yes,yes,yes,no,low
9,10,3.798,Polygon,CIV,District Autonome D'Abidjan,-4.119589,5.572136,-,ha,0.000,...,0.000,0.000,0.000,0.000,9,yes,yes,yes,no,low


<br>

#### If you do **not** want to generate GeoIDs for your geometries, 
##### export your risk table as CSV with the following code. Otherwise, please jump to the next code section.

In [11]:
# Create a copy of the DataFrame for export purposes
df_for_export = df_w_risk.copy()

# Add 'geo_id_column' holding 'na' values in the copy, only if it's not present
if geo_id_column not in df_for_export.columns:
    df_for_export[geo_id_column] = 'na'

# Define the output file path
output_file = out_directory / 'whisp_output_table_w_risk.csv'  # Edit as required

# Export the modified copy to CSV
df_for_export.to_csv(output_file, index=False)

# Print the output file location
print("Output CSV: ", output_file)


Output CSV:  /home/sepal-user/module_results/whisp/whisp_output_table_w_risk.csv


<br>

#### If you do want to generate GeoIDs for your geometries, 
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- If processing doesn't complete, just rerun this cell and it should pickup from where you left.
- If registering many features you can also click dollar sign in bottom right of Sepal and under 'Sessions' heading extend the length as needed.

In [12]:
output_file=out_directory/'whisp_output_table_w_risk.csv' # edit as required
df_w_risk.to_csv(output_file)
print("output csv: ", output_file)

register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="geoid",
    output_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
    join_id_column="system:index",
    override_checks=False,
    remove_temp_csv=False,
    debug=False)

output csv:  /home/sepal-user/module_results/whisp/whisp_output_table_w_risk.csv
Chosen path for temp lookup csv:/home/sepal-user/module_results/whisp/temp_geo_id_lookup.csv 

Check passed: feature collection and table same size: 31 rows
All geo ids present in lookup csv
Done 



##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [13]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv=out_directory/"whisp_output_table_w_risk.csv",
        geo_id_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        overwrite=False,
        drop_geo=False,
        debug=True)

new csv:  /home/sepal-user/module_results/whisp/whisp_output_table_w_risk_w_geo_id.csv


##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [14]:
# remove_column_from_csv(
#     csv_file="whisp_output_table_w_risk_w_geo_id.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )